In [111]:
import pandas as pd
import os
import string
import re
from nltk.stem import WordNetLemmatizer
import nltk
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [112]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [131]:
def clean_text(text):
    text_nopunct = "".join([char for char in text.lower() if char not in string.punctuation])
    tokens = re.split('\W+', text_nopunct) #tokenize
    words_without_stopwords = [word for word in tokens if word not in stopword] #remove stopwords from tokens
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in words_without_stopwords]

In [114]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

In [115]:
df.shape

(100000, 15)

In [116]:
df[df['Subject'].isna()].shape

(5364, 15)

In [117]:
df[df['content'].isna()].shape

(402, 15)

##### Right now just removing the nan value, may look into this later for do preprocessing

In [128]:
df = df[df['Subject'].notna()]
df = df[df['content'].notna()]
df[df['content'].isna()].shape

(0, 15)

In [142]:

dfEmail = df[['Subject', 'content']]
dfEmail['content_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))
dfEmail['Subject_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [144]:
dfEmail.head()

,Subject,content,content_lemma
1,<15464986.1075855378456.JavaMail.evans@thyme>,Traveling to have a business meeting takes the...,"[travel, business, meet, take, fun, trip, espe..."
2,<24216240.1075855687451.JavaMail.evans@thyme>,test successful. way to go!!!,"[test, successful, way, go]"
4,<30922949.1075863688243.JavaMail.evans@thyme>,Let's shoot for Tuesday at 11:45.,"[let, shoot, tuesday, 1145]"
5,<30965995.1075863688265.JavaMail.evans@thyme>,"Greg, How about either next Tuesday or Thursda...","[greg, either, next, tuesday, thursday, phillip]"
7,<17189699.1075863688308.JavaMail.evans@thyme>,any morning between 10 and 11:30,"[morning, 10, 1130]"


In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [153]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer=clean_text)

In [154]:
data_matrix=vectorizer.fit_transform(dfEmail['content'])
print("\n Feature names Identified :\n")
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [162]:
print(vectorizer.get_feature_names())

{'smooth_idf': True, 'max_features': None, 'binary': False, 'norm': 'l2', 'encoding': 'utf-8', 'preprocessor': None, 'use_idf': True, 'input': 'content', 'min_df': 1, 'tokenizer': None, 'strip_accents': None, 'ngram_range': (1, 1), 'lowercase': True, 'sublinear_tf': False, 'analyzer': <function clean_text at 0x7f27e1491bf8>, 'decode_error': 'strict', 'max_df': 1.0, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'dtype': <class 'numpy.float64'>, 'vocabulary': None, 'stop_words': 'english'}


In [163]:
print(data_matrix)

  (0, 277790)	0.049289919352570814
  (0, 205582)	0.07134563787631636
  (0, 263026)	0.13274178135757722
  (0, 137320)	0.11938494808345629
  (0, 173327)	0.134592918286781
  (0, 83374)	0.1462858013324741
  (0, 260804)	0.2896478577394565
  (0, 241425)	0.1350812847455288
  (0, 145479)	0.1268466745696565
  (0, 228340)	0.0888451244088858
  (0, 74823)	0.11100428773595755
  (0, 145184)	0.054074833732442186
  (0, 269513)	0.1288253159070854
  (0, 138240)	0.10908114332178932
  (0, 116080)	0.09077325218762469
  (0, 271909)	0.1010065737954542
  (0, 246928)	0.11010671517909407
  (0, 80332)	0.08233001418094846
  (0, 200644)	0.07953419975954631
  (0, 283114)	0.08932665295582067
  (0, 292707)	0.09631960339748076
  (0, 236340)	0.1434537457953647
  (0, 220132)	0.08563073084685194
  (0, 263903)	0.08725716758853988
  (0, 231864)	0.18166477958775254
  :	:
  (94463, 281017)	0.05226618586354107
  (94463, 171058)	0.07873301618580102
  (94463, 302738)	0.053644605118566446
  (94463, 103831)	0.159160344126019
  (9

In [171]:
data = vectorizer.fit_transform(['Traveling to have a business meeting takes th'])

In [172]:
print(data)

  (0, 3)	0.4472135954999579
  (0, 2)	0.4472135954999579
  (0, 1)	0.4472135954999579
  (0, 0)	0.4472135954999579
  (0, 4)	0.4472135954999579
